<div class="alert alert-box alert-info">
    <h2 style="margin:0px">Importing Libraries</h2>
 </div>

In [55]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
import matplotlib.pyplot as plt
import warnings
import pickle as pkl
warnings.filterwarnings("ignore")
#ignoring the warnings

<div class="alert alert-box alert-info">
    <h2 style="margin:0px">Loading Dataset</h2>
 </div>

In [53]:
dataset=load_boston()

In [54]:
X=dataset["data"]

In [46]:
y=dataset["target"]

In [47]:
X.shape

(506, 13)

<div class="">
    <h2 style="margin:200px"></h2>
 </div>

<div class="alert alert-box alert-info">
    <h2 style="margin:0px">Linear Regression</h2>
 </div>

In [62]:
class LinearRegression:
    def __init__(self,learning_rate=0.1,iteration=5000):
        """LinearRegression implementation 
        Learning_rate = rate of gradient descent reaching minima
        iteration = number of iteration - (can only be integer)"""
        self.itr=iteration
        self.lr=learning_rate
        self.weights=None
        self.bias=None
        self.samples=0
        self.features=0
        
    def _init_params(self):
        #initializing the parameters
        self.weights=np.zeros(self.features)
        self.bias=0
        
    def _calculate_cost(self, y, y_pred):
        return (1 / (2*self.samples)) * np.sum(np.square(y_pred-y))
        #cost function - repeat untill convergence
        
    def _get_prediction(self,X):
        return np.dot(X,self.weights)+self.bias
        #get the prediction for the current values of weights and bias
        
    def _update_params(self,dw,db):
        self.weights-=self.lr*dw
        self.bias-=self.lr*db
        #lr - learning rate
        
    def _get_gradient(self,X,y,y_pred):
        error=y_pred-y
        dw=(1/self.samples)*np.dot(X.T,error)
        db=(1/self.samples)*np.sum(error)
        #partial derivative
        return dw,db
    
    def fit(self,X,y):
        #fit function trains the model
        self.samples,self.features=X.shape
        self._init_params()
        try:
            for i in range(self.itr):
                y_pred=self._get_prediction(X)
                cost=self._calculate_cost(y,y_pred)
                dw,db=self._get_gradient(X,y,y_pred)
                self._update_params(dw,db)
                if i % 100 == 0:
                    print(f"The Cost in iteration {i}----->{cost}") 
        except ValueError:
            print("nan value or some unidentified values found")
        except Exception as e:
            print(f"Exception found {e}")      
    def predict(self,X):
        return self._get_prediction(X)
        #calls _get_prediction method
        
    def save_model(self):
        try:
            file = open('Linear_regression.pkl', 'wb')
            pkl.dump(self,file)
            file.close()
            #save the model for deployment or for future training
        except Exception as e:
            print(f"Exception found {e}")


<div class="">
    <h2 style="margin:400px"></h2>
 </div>

<div class="alert alert-box alert-info">
    <h2 style="margin:0px">Ridge (L2 Regularization)</h2>
 </div>

In [111]:
class RidgePenalty:
    """Lasso Regularization class 
    """
    def __init__(self,alpha):
        self.alpha=alpha
    def __call__(self,w):
        return self.alpha*np.sum(np.square(w))
    def derivation(self,w):
        return self.alpha*2*w

class Ridge(LinearRegression):
    def __init__(self,alpha,learning_rate=0.1,iteration=5000):
        """Ridge Regression implementation - inherits Linear Regression
        alpha - hyperparameter
        Learning_rate = rate of gradient descent reaching minima
        iteration = number of iteration - (can only be integer)"""
        self.regularization=Ridgepenalty(alpha)
        #initialize the Ridge penalty        
        super().__init__(learning_rate,iteration)
        #initialize the super class i.e LinearRegression

    def _calculate_cost(self, y, y_pred):
        return super()._calculate_cost(y,y_pred)+self.regularization(self.weights)
        #cost functon
    
    def fit(self,X,y):
        try:
            self.samples,self.features=X.shape
            super()._init_params()
            for i in range(self.itr):
                y_pred=super()._get_prediction(X)
                cost=self._calculate_cost(y,y_pred)
                dw,db=super()._get_gradient(X,y,y_pred)
                dw+=self.regularization.derivation(self.weights)
                self._update_params(dw,db)
                if i % 100 ==0:
                    print(f"The Cost in iteration {i}----->{cost}")
                    
        except ValueError:
            print("nan value or some unidentified values found")
        except Exception as e:
            print(f"Exception found {e}")
                
    def predict(self,X):
        return super().predict(X)

<div class="alert alert-box alert-info">
    <h2 style="margin:0px">Lasso (L1 Regularization)</h2>
 </div>

In [64]:
class LassoPenalty:
    """Lasso Regularization class """
    def __init__(self,alpha):
        self.alpha=alpha
    def __call__(self,w):
        return self.alpha*np.sum(np.abs(w))
    def derivation(self,w):
        return self.alpha*np.sign(w)

class Lasso(LinearRegression):
    def __init__(self,alpha,learning_rate=0.1,iteration=5000):
        """Lasso Regression implementation - inherits Linear Regression
        alpha - hyperparameter
        Learning_rate = rate of gradient descent reaching minima
        iteration = number of iteration - (can only be integer)
        """
        self.regularization=Lassopenalty(alpha)
        #initialize the Ridge penalty
        
        super().__init__(learning_rate,iteration)
        #initialize the super class i.e LinearRegression

    def _calculate_cost(self, y, y_pred):
        return super()._calculate_cost(y,y_pred)+self.regularization(self.weights)
    #cost function 
    
    def fit(self,X,y):
        self.samples,self.features=X.shape
        super()._init_params()
        for i in range(self.itr):
            y_pred=super()._get_prediction(X)
            cost=self._calculate_cost(y,y_pred)
            dw,db=super()._get_gradient(X,y,y_pred)
            dw+=self.regularization.derivation(self.weights)
            self._update_params(dw,db)
            if i % 100 ==0:
                print(f"The Cost in iteration {i}----->{cost}")
                
    def predict(self,X):
        return super().predict(X)

<div class="alert alert-box alert-info">
    <h2 style="margin:0px">ElasticNet (L1 and L2 Regularization) </h2>
 </div>

In [132]:

class ElasticNetPenalty(LassoPenalty,RidgePenalty):
    """ElasticNet Regularization class -inherits Lasso and Ridge penalty"""
    def __init__(self,l1_ratio,alpha):
        self.l1_ratio=l1_ratio
        self.Lasso=LassoPenalty(alpha)
        self.Ridge=RidgePenalty(alpha)
    def cost(self,w):
        return (self.l1_ratio)*(self.Lasso(w)) +(1-self.l1_ratio)*(self.Ridge(w))
    def derivation(self,w):
        return (self.l1_ratio)*(self.Lasso.derivation(w)) +(1-self.l1_ratio)*(self.Ridge.derivation(w))
    
    
class ElasticNet(LinearRegression):
    def __init__(self,alpha,l1_ratio, learning_rate=0.01, iteration=3000):
        """Elastic Net Regression implementation - inherits Linear Regression
        alpha - hyperparameter
        L1_ratio- ratio of penalty
        Learning_rate = rate of gradient descent reaching minima
        iteration = number of iteration - (can only be integer) """
        self.regularization=ElasticNetPenalty(alpha,l1_ratio)
        #initialize the Elastic Net penalty
        
        super().__init__(learning_rate, iteration)
        #initialize the super class i.e LinearRegression
        
    def _calculate_cost(self, y, y_pred):
        return super()._calculate_cost(y, y_pred)+self.regularization.cost(self.weights)
    #cost function
    
    def fit(self,X,y):
        self.samples,self.features=X.shape
        self._init_params()
        for i in range(self.itr):
            y_pred=super()._get_prediction(X)
            cost=self._calculate_cost(y,y_pred)
            dw,db=super()._get_gradient(X,y,y_pred)
            dw+=self.regularization.derivation(self.weights)
            self._update_params(dw,db)
            if i % 100 ==0:
                print(f"The Cost in iteration {i}----->{cost}")
                
    def predict(self,X):
        return super().predict(X)

<div class="">
    <h2 style="margin:150px"></h2>
 </div>

<div class="alert alert-box alert-info">
    <h2 style="margin:0px">Test Train split</h2>
 </div>

In [106]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1)

<div class="alert alert-box alert-info">
    <h2 style="margin:0px">Applying Standardization</h2>
 </div>

In [107]:
from sklearn.preprocessing import StandardScaler
st=StandardScaler()
X_train=st.fit_transform(X_train)
X_test=st.transform(X_test)

<div class="">
    <h2 style="margin:300px"></h2>
 </div>

<div class="alert alert-box alert-info">
    <h2 style="margin:0px">Implementing Linear Regression regression</h2>
 </div>

In [112]:
from sklearn.metrics import r2_score
LR=LinearRegression(0.1,3000)
LR.fit(X_train,y_train)
y_predict=LR.predict(X_test)
r2_score(y_test,y_predict)
#calculating R2 Score

The Cost in iteration 0----->297.37484615384614
The Cost in iteration 100----->11.67190100608827
The Cost in iteration 200----->11.564670476056179
The Cost in iteration 300----->11.544346117022767
The Cost in iteration 400----->11.539036888946223
The Cost in iteration 500----->11.53757302292186
The Cost in iteration 600----->11.537166351224284
The Cost in iteration 700----->11.537053259251122
The Cost in iteration 800----->11.537021804943727
The Cost in iteration 900----->11.537013056381053
The Cost in iteration 1000----->11.537010623088031
The Cost in iteration 1100----->11.53700994630063
The Cost in iteration 1200----->11.537009758061398
The Cost in iteration 1300----->11.53700970570521
The Cost in iteration 1400----->11.537009691143046
The Cost in iteration 1500----->11.537009687092775
The Cost in iteration 1600----->11.537009685966249
The Cost in iteration 1700----->11.537009685652919
The Cost in iteration 1800----->11.53700968556577
The Cost in iteration 1900----->11.5370096855415

0.8284494153200391

<div class="">
    <h2 style="margin:50px"></h2>
 </div>

<div class="alert alert-box alert-info">
    <h2 style="margin:0px">Implementing Ridge regression</h2>
 </div>

In [122]:
RID=Ridge(0.1,0.1,3000)
RID.fit(X_train,y_train)
y_predict=RID.predict(X_test)
r2_score(y_test,y_predict)
#calculating R2 Score

The Cost in iteration 0----->297.37484615384614
The Cost in iteration 100----->15.120786743885096
The Cost in iteration 200----->15.120095832846003
The Cost in iteration 300----->15.12009380384473
The Cost in iteration 400----->15.120093794922692
The Cost in iteration 500----->15.120093794880631
The Cost in iteration 600----->15.120093794880432
The Cost in iteration 700----->15.120093794880432
The Cost in iteration 800----->15.120093794880432
The Cost in iteration 900----->15.120093794880434
The Cost in iteration 1000----->15.120093794880432
The Cost in iteration 1100----->15.120093794880432
The Cost in iteration 1200----->15.120093794880432
The Cost in iteration 1300----->15.120093794880432
The Cost in iteration 1400----->15.120093794880432
The Cost in iteration 1500----->15.120093794880432
The Cost in iteration 1600----->15.120093794880432
The Cost in iteration 1700----->15.120093794880432
The Cost in iteration 1800----->15.120093794880432
The Cost in iteration 1900----->15.120093794

0.8395354974625773

<div class="">
    <h2 style="margin:80px"></h2>
 </div>

<div class="alert alert-box alert-info">
    <h2 style="margin:0px">Implementing Lasso regression</h2>
 </div>

In [124]:
LAS=Lasso(0.001,0.1,3000)
LAS.fit(X_train,y_train)
y_predict=LAS.predict(X_test)
r2_score(y_test,y_predict)
#calculating R2 Score

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 0
The Cost in iteration 0----->297.37484615384614
The Cost in iteration 100----->11.692540906723025
The Cost in iteration 200----->11.586345947111166
The Cost in iteration 300----->11.566492612686854
The Cost in iteration 400----->11.561384650016578
The Cost in iteration 500----->11.559976757478516
The Cost in iteration 600----->11.559585646519556
The Cost in iteration 700----->11.559476882111507
The Cost in iteration 800----->11.559446631437059
The Cost in iteration 900----->11.559438217647685
The Cost in iteration 1000----->11.559435877467282
The Cost in iteration 1100----->11.559435226577888
The Cost in iteration 1200----->11.559435045541834
The Cost in iteration 1300----->11.559434995189111
The Cost in iteration 1400----->11.559434981184188
The Cost in iteration 1500----->11.559434977288907
The Cost in iteration 1600----->11.559434976205488
The Cost in iteration 1700----->11.559434975904146
The Cost in iteration 1800----->11.559434975820334


0.8285684447316505

<div class="">
    <h2 style="margin:80px"></h2>
 </div>

<div class="alert alert-box alert-info">
    <h2 style="margin:0px">Implementing ElasticNet regression</h2>
 </div>

In [139]:
ELN=ElasticNet(0.1,0.5,0.001,3000)
ELN.fit(X_train,y_train)
y_predict=ELN.predict(X_test)
r2_score(y_test,y_predict)
#calculating R2 Score

The Cost in iteration 0----->297.37484615384614
The Cost in iteration 100----->238.55724915526173
The Cost in iteration 200----->196.5167857619539
The Cost in iteration 300----->163.7835403569911
The Cost in iteration 400----->137.53070692882102
The Cost in iteration 500----->116.25817046921699
The Cost in iteration 600----->98.9474792872051
The Cost in iteration 700----->84.83508525561747
The Cost in iteration 800----->73.31821957291808
The Cost in iteration 900----->63.91301557224504
The Cost in iteration 1000----->56.228372237412245
The Cost in iteration 1100----->49.947068020790695
The Cost in iteration 1200----->44.811249157916606
The Cost in iteration 1300----->40.61099804670377
The Cost in iteration 1400----->37.175223526973454
The Cost in iteration 1500----->34.364353638801525
The Cost in iteration 1600----->32.06444810328093
The Cost in iteration 1700----->30.18243893171322
The Cost in iteration 1800----->28.642273059356533
The Cost in iteration 1900----->27.381971836583634
Th

0.7882085812445028